# CIFAR-10 Image Classification 

In [1]:
# Import Libraries

import numpy
import time
import calendar
from six.moves import cPickle as pickle
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

##### General parameters

In [2]:
# General Parameters

imageSize = 32  
channels = 3    
classes = 10    
trainingDataSize = 50000   
testDataSize = 10000        
trainigDataFiles = ('./dataset/cifar-10-batches-py/data_batch_1', './dataset/cifar-10-batches-py/data_batch_2', './dataset/cifar-10-batches-py/data_batch_3', './dataset/cifar-10-batches-py/data_batch_4','./dataset/cifar-10-batches-py/data_batch_5') 
testDataFile = './dataset/cifar-10-batches-py/test_batch' 
pcaFileName = 'pca' 
svmFileName = 'svm' 

##### HOG Parameters

In [3]:
# HOG Parameters

winSize = imageSize
blockSize = 12
blockStride = 4
cellSize = 4
nbins = 18
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = True
nlevels = 64
signedGradient = True
hog = cv2.HOGDescriptor((winSize,winSize),(blockSize, blockSize),(blockStride,blockStride),(cellSize,cellSize),nbins,derivAperture, winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels,signedGradient)


##### SVM parameters

In [4]:
# SVM parameters

svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_RBF)
svm.setC(0.1)
svm.setGamma(0.1)

##### PCA parameters

In [5]:
# PCA parameters

pcaDim = 3000

##### Load the data from the given filename

In [6]:
######################################################################################
def loadData(filename):
    '''
    Load the data from the given filename

    Parameters:
    -----------
    filename: string
        The name of the file containing the data to load

    Returns:
    --------
    theSet['data']:     array of images
    theSet['labels']:   array of labels
    '''
    f = open(filename, 'rb')
    theSet = pickle.load(f,encoding='latin1')
    f.close()

    return theSet['data'], theSet['labels']
#####################################################################################

##### Convert the images from CIFAR-10 format, to an array of 10000 images each is 32 X 32 X 3 size

In [7]:
#####################################################################################
def convertImages(origImages):
    '''
    Convert the images from CIFAR-10 format, to an array of 10000 images each is 32 X 32 X 3 size

    Parameters:
    -----------
    origImages: array
        array of images in the CIFAR-10 format

    Returns:
    --------
    images:     array of images each in 32 X 32 X 3 size
    '''
    images = np.reshape(origImages,(-1, channels, imageSize, imageSize))
    images = np.transpose(images, (0,2,3,1))

    return images
#####################################################################################

##### Load the test data

In [8]:
######################################################################################
def loadTestData(filename):
    '''
    Load the test data

    Parameters:
    -----------
    filename: string
        The name of the file containing the test data

    Returns:
    --------
    testImages: array of images of the test data
    testLabels: array of labels of the test data
    '''

    origTestImages, testLabels = loadData(filename)
    testImages = convertImages(origTestImages)

    return testImages, testLabels
######################################################################################

##### Load all the training data from all files

In [9]:
######################################################################################
def loadTrainingData(filenames):
    '''
    Load all the training data from all files

    Parameters:
    -----------
    filenames: array of string
        An array The name of the file containing the data to load

    Returns:
    --------
    trainingImages: array of the training set images
    trainingLabels: array of the training set labels
    '''

    #Pre-allocate the arrays
    trainingImages = np.zeros(shape=[trainingDataSize, imageSize, imageSize, channels], dtype=numpy.uint8)
    trainingLabels = np.zeros(shape=[trainingDataSize], dtype=int)

    start=0
    for fileName in filenames:
        origImages, labels = loadData(fileName)
        images = convertImages(origImages)

        numOfImages = len(images);
        end = start + numOfImages;
        trainingImages[start:end, :] = images
        trainingLabels[start:end] = labels
        start = end

    return trainingImages, trainingLabels
#####################################################################################

##### Returns the current time in seconds since EPOC

In [10]:
#####################################################################################
def currentTime():
    '''
    Returns the current time in seconds since EPOC
    Used to measure how much time each phase took

    Returns:
    --------
    the current time in second since EPOC
    '''

    return calendar.timegm(time.gmtime())
#####################################################################################

##### Calculate the HOG descriptors of the given images

In [11]:
#####################################################################################
def calcHOG(images):
    '''
    Calculate the HOG descriptors of the given images

    Parameters:
    -----------
    images: an array of images
        The images to which a HOG calculation should be applied

    Returns:
    --------
    hogDescriptors: an array of HOG vectors, 5832 components each
    '''

    hogDescriptors = []
    for image in images:
        hogDescriptors.append( hog.compute(image) )

    hogDescriptors = np.squeeze(hogDescriptors)
    return hogDescriptors
#####################################################################################

#### Algorithm Implementation

##### Training Phase

###### First load the data into two arrays:

In [12]:
from google.colab import drive
drive.mount('/content/Drive', force_remount=True)

Mounted at /content/Drive


In [13]:
!mkdir -p "/content/dataset"
!tar -xzf "/content/Drive/MyDrive/CIFAR-10/cifar-10-python.tar.gz" -C "/content/dataset"

In [14]:
print("Loading the training set..."),
tik = currentTime()
trainingImages, trainingLabels = loadTrainingData(trainigDataFiles)
print("Took: " + str(currentTime()-tik) + " sec" )

Loading the training set...
Took: 0 sec


###### Create a HOG descriptor from these images

In [15]:
print("Creating HOG descriptors from the training set..."),
tik = currentTime()
trainHogDescriptors = calcHOG(trainingImages)
print("Took: " + str(currentTime() - tik) + " sec")

Creating HOG descriptors from the training set...
Took: 6 sec


In [18]:
print(type(trainHogDescriptors))
print(trainHogDescriptors.shape)

<class 'numpy.ndarray'>
(50000, 5832)


###### Reduce the dimension of the HOG descriptors to 3000

In [19]:
print("Reducing the dimension of the HOG descriptors to " + str(pcaDim) + "..."),
tik = currentTime()
pca = PCA(pcaDim)
trainHogProjected = pca.fit_transform(trainHogDescriptors)
print("Took: " + str(currentTime() - tik) + " sec")

Reducing the dimension of the HOG descriptors to 3000...
Took: 381 sec


###### Save it as a pca file

In [ ]:
print("Save it as a PCA file..."),
tik = currentTime()
pcaFile = open(pcaFileName, 'wb')
pickle.dump(pca, pcaFile)
pcaFile.close()
print("Took: " + str(currentTime() - tik) + " sec")

Save it as a PCA file...
Took: 0 sec


###### Train the SVM model using the reduced HOG descriptor

In [ ]:
print("Training the SVM model using the reduced HOG descriptor..."),
tik = currentTime()
svm.train(np.asarray(trainHogProjected), cv2.ml.ROW_SAMPLE, np.asarray(trainingLabels))
svm.save(svmFileName)
print("Took: " + str(currentTime() - tik) + " sec")

Training the SVM model using the reduced HOG descriptor...
Took: 1386 sec


##### Testing phase

###### Load the test set

In [ ]:
print("Loading the test set..."),
tik = currentTime()
testImages, testLabels = loadTestData(testDataFile)
print("Took: " + str(currentTime() - tik) + " sec")

Loading the test set...
Took: 0 sec


###### Create HOG descriptors from the test set

In [ ]:
print("Creating HOG descriptors from the test set..."),
tik = currentTime()
testHogDescriptors = calcHOG(testImages);
print("Took: " + str(currentTime() - tik) + " sec")

Creating HOG descriptors from the test set...
Took: 1 sec


###### Reduce the dimension of the HOG descriptors

In [ ]:
print("Reducing the dimension of the HOG descriptors to " + str(pcaDim) + "..."),
tik = currentTime()
testHogProjected = pca.transform(testHogDescriptors)
print("Took: " + str(currentTime() - tik) + " sec")

Reducing the dimension of the HOG descriptors to 3000...
Took: 4 sec


###### Classify the test set

In [ ]:
print("Classifying the test set..."),
tik = currentTime()
testResponse = svm.predict(np.asarray(testHogProjected))[1].ravel()
print("Took: " + str(currentTime() - tik) + " sec")

Classifying the test set...
Took: 361 sec


###### Calculate the confusion matrix

In [ ]:
print ("Confusion matrix:")
print ("=================")
confusionMatrix = confusion_matrix(testLabels, testResponse)
print(confusionMatrix)

Confusion matrix:
[[780   7  49  24  22  10  10   3  71  24]
 [ 24 834   8  13  13   5  14   4  30  55]
 [ 88   8 574  82  86  67  41  24  22   8]
 [ 36  10 103 530  61 154  47  27  17  15]
 [ 25  12  81  89 640  46  34  51  17   5]
 [ 17   6  99 214  47 527  31  47   5   7]
 [ 17  17  39  76  42  39 750   3  11   6]
 [ 11   5  52  51  59  53   7 732  13  17]
 [ 62  26  14  16  15   9  10   7 816  25]
 [ 33  51  16  17  10  11   7  11  20 824]]


###### Calculate accuracy

In [ ]:
print ("======================================")
accuracy = (np.asarray(testLabels) == testResponse).mean()
print("Percentage Accuracy: %.2f %%" % (accuracy*100))
print ("======================================")

Percentage Accuracy: 70.07 %
